In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'dataset/train_mtcnn_eq'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 26880 images belonging to 480 classes.
Found 6720 images belonging to 480 classes.


In [5]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
import json
import pandas as pd
import os
import tensorflow as tf

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

os.makedirs('finetune', exist_ok=True)
history_path = 'training_log.csv'
def append_history(history, file_name):
    history_df = pd.DataFrame(history.history)
    if os.path.isfile(file_name):
        history_df.to_csv(file_name, mode='a', header=False, index=False)
    else:
        history_df.to_csv(file_name, index=False)

ckpt_index = 'c2'
checkpoint = ModelCheckpoint(f'finetune/finetuned_model_{ckpt_index}.h5', monitor='val_loss', save_best_only=True, mode='min')
csv_logger = CSVLogger('training_log.csv', append=True)

saved_model_path = 'saved_model_c1'
loaded_model = tf.saved_model.load(saved_model_path)
facenet_model = tf.keras.Sequential([loaded_model])
facenet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = facenet_model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[checkpoint, csv_logger, reduce_lr]
)

append_history(history, history_path)

history_df = pd.read_csv(history_path)
tf.saved_model.save(facenet_model, f'saved_model_{ckpt_index}')

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x000001FF9D4D07C8>

In [ ]:
history_df = pd.read_csv(history_path)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_df['accuracy'], label='train accuracy')
plt.plot(history_df['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_df['loss'], label='train loss')
plt.plot(history_df['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.savefig(f'training_performance_{ckpt_index}.png')
plt.show()